This tutorial describes how to evaluate new rules applicable to the assets and derivatives data.

In [ ]:
from arelle import ModelManager, Cntlr, ModelFormulaObject, ModelXbrl, ViewFileFormulae, XbrlConst, ViewFileRenderedGrid
from arelle import RenderingEvaluator 

In [ ]:
import pandas as pd
import numpy as np
import math
from os import listdir
from os.path import join, isfile
import re
from src import Evaluator
import logging
import data_patterns

In [ ]:
DECIMALS = 0
RULES_PATH = join('..', 'solvency2-rules')
INSTANCES_DATA_PATH = join('..', 'data', 'instances', '...') #path of folder with converted xbrl-instance data
TEST_DATA_PATH = join('..', 'tests', 'data', 'demo') #path of folder with demo data
RESULTS_PATH = join('..', 'results')
DATA_PATH = join('..', 'data')
logging.basicConfig(filename = join(RESULTS_PATH, 'rules.log'),level = logging.INFO, 
                    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

### Import rules

We start with importing the new rules applicable to the assets and derivatives data. There are several sets of rules applicable to different templates:
* S.06.02.01 (Information on positions held)
* S.06.02.02 (Information on assets)
* S.06.02.01 (Information on positions held) and S.06.02.02 (Information on assets)
* S.08.01.01.01 (Information on positions held) and S2.08.01.01.02 (Information on derivatives)
* S.08.01.01.02 (Information on derivatives)

#### S06

In [ ]:
dfr_s06 = pd.read_excel(join(RULES_PATH,'S2_06_02.xlsx'), engine='openpyxl')
dfr_s06_2 = pd.read_excel(join(RULES_PATH,'S2_06_02_01_02.xlsx'), engine='openpyxl')
dfr_s06_1 = pd.read_excel(join(RULES_PATH,'S2_06_02_01_01.xlsx'), engine='openpyxl')

#Capitalize row-column references:
column_replace = set([column for sublist in [row for row in dfr_s06['pandas ex'].str.findall(r'c\d\d\d\d')] for column in sublist])
for ref in column_replace:
    dfr_s06.replace(to_replace=ref, value=ref.capitalize(), inplace=True, regex=True)
column_replace = set([column for sublist in [row for row in dfr_s06_2['pandas ex'].str.findall(r'c\d\d\d\d')] for column in sublist])
for ref in column_replace:
    dfr_s06_2.replace(to_replace=ref, value=ref.capitalize(), inplace=True, regex=True)
column_replace = set([column for sublist in [row for row in dfr_s06_1['pandas ex'].str.findall(r'c\d\d\d\d')] for column in sublist])
for ref in column_replace:
    dfr_s06_1.replace(to_replace=ref, value=ref.capitalize(), inplace=True, regex=True)

#### S08

In [ ]:
dfr_s08 = pd.read_excel(join(RULES_PATH,'S2_08_01_01.xlsx'), engine='openpyxl')
dfr_s08_2 = pd.read_excel(join(RULES_PATH,'S2_08_01_01_02.xlsx'), engine='openpyxl')

#Capitalize row-column references:
column_replace = set([column for sublist in [row for row in dfr_s08['pandas ex'].str.findall(r'c\d\d\d\d')] for column in sublist])
for ref in column_replace:
    dfr_s08.replace(to_replace=ref, value=ref.capitalize(), inplace=True, regex=True)
column_replace = set([column for sublist in [row for row in dfr_s08_2['pandas ex'].str.findall(r'c\d\d\d\d')] for column in sublist])
for ref in column_replace:
    dfr_s08_2.replace(to_replace=ref, value=ref.capitalize(), inplace=True, regex=True)

### Import templates

Next we import the reporting data. In the tutorial 'Convert XBRL-instances to CSV, HTML and pickles' the XBRL-instances are converted to pickle files per template. The pickle files are written to the data/instances folder. We import these pickle files. We merge dataframes for the sets of rules that are applicable to two templates. For the sake of simplicity we only import the Quarterly Solvency II reporting Solo (QRS) templates.

#### S06

Import data and make index unique if necessary

In [ ]:
# df_s06_1 = pd.read_pickle(join(INSTANCES_DATA_PATH,'S.06.02.01.01.pickle')).fillna(0).reset_index()
# df_s06_1['S.06.02.01.01,C0040A'] = df_s06_1['S.06.02.01.01,C0040']
# listt=list(df_s06_1['S.06.02.01.01,C0040A'])
# for i in listt:
#     lenn = len(df_s06_1[df_s06_1['S.06.02.01.01,C0040A']==i])
#     if lenn > 1:
#         list_ind = list(df_s06_1.loc[df_s06_1['S.06.02.01.01,C0040A']==i].index)
#         temp = 0
#         for j in list_ind[1:]:
#             temp=temp+1
#             df_s06_1['S.06.02.01.01,C0040A'].iloc[j] = df_s06_1['S.06.02.01.01,C0040A'].iloc[j] + '_' + str(temp)
# df_s06_1 = df_s06_1.set_index(['entity', 'period', 'S.06.02.01.01,C0040A'])

In [ ]:
# df_s06_2 = pd.read_pickle(join(INSTANCES_DATA_PATH, 'S.06.02.01.02.pickle')).fillna(0).reset_index()
# df_s06_2 = df_s06_2.set_index(['entity', 'period', 'S.06.02.01.02,C0040'])
# df_s06_2['S.06.02.01.02,C0040'] = df_s06_2.index.get_level_values(2)

In [ ]:
# df_s06 = pd.merge(pd.read_pickle(join(INSTANCES_DATA_PATH,'S.06.02.01.01.pickle')).reset_index(),pd.read_pickle(join(INSTANCES_DATA_PATH, 'S.06.02.01.02.pickle')).reset_index(),how='inner', left_on=['entity','period','S.06.02.01.01,C0040'], right_on=['entity','period','S.06.02.01.02,C0040']).set_index(['entity', 'period', 'S.06.02.01.01,C0040'])
# df_s06 = df_s06.fillna(0).reset_index()
# df_s06['S.06.02.01.02,C0040A'] = df_s06['S.06.02.01.02,C0040']
# listt=list(df_s06['S.06.02.01.02,C0040A'])
# for i in listt:
#     lenn = len(df_s06[df_s06['S.06.02.01.02,C0040A']==i])
#     if lenn > 1:
#         list_ind = list(df_s06.loc[df_s06['S.06.02.01.02,C0040A']==i].index)
#         temp = 0
#         for j in list_ind[1:]:
#             temp=temp+1
#             df_s06['S.06.02.01.02,C0040A'].iloc[j] = df_s06['S.06.02.01.02,C0040A'].iloc[j] + '_' + str(temp)
# df_s06 = df_s06.set_index(['entity', 'period', 'S.06.02.01.02,C0040A'])

In this tutorial we work with dummy data in order to show results

In [ ]:
df_s06_1 = pd.read_pickle(join(TEST_DATA_PATH,'S.06.02.01.01.pickle')).reset_index() #Import demo pickles
df_s06_2 = pd.read_pickle(join(TEST_DATA_PATH,'S.06.02.01.02.pickle')).reset_index() #Import demo pickles
df_s06 = pd.merge(df_s06_1,df_s06_2,how='inner', left_on=['entity','period','S.06.02.01.01,C0040'], right_on=['entity','period','S.06.02.01.02,C0040']).set_index(['entity', 'period', 'S.06.02.01.01,C0040'])
df_s06_2 = df_s06_2.set_index(['entity', 'period', 'S.06.02.01.02,C0040'])
df_s06_1 = df_s06_1.set_index(['entity', 'period', 'S.06.02.01.01,C0040'])
df_s06_2['S.06.02.01.02,C0040'] = df_s06_2.index.get_level_values(2)
df_s06['S.06.02.01.01,C0040'] = df_s06.index.get_level_values(2)

#### S08

Import data and make index unique if necessary

In [ ]:
# df_s08_2 = pd.read_pickle(join(INSTANCES_DATA_PATH, 'S.08.01.01.02.pickle')).fillna(0).reset_index()
# df_s08_2 = df_s08_2.set_index(['entity', 'period', 'S.08.01.01.02,C0040'])
# df_s08_2['S.08.01.01.02,C0040'] = df_s08_2.index.get_level_values(2)

In [ ]:
# df_s08 = pd.merge(pd.read_pickle(join(INSTANCES_DATA_PATH,'S.08.01.01.01.pickle')).reset_index(),pd.read_pickle(join(INSTANCES_DATA_PATH, 'S.08.01.01.02.pickle')).reset_index(),how='inner', left_on=['entity','period','S.08.01.01.01,C0040'], right_on=['entity','period','S.08.01.01.02,C0040']).set_index(['entity', 'period', 'S.08.01.01.01,C0040'])
# df_s08 = df_s08.fillna(0).reset_index()
# df_s08['S.08.01.01.02,C0040A'] = df_s08['S.08.01.01.02,C0040']
# listt=list(df_s08['S.08.01.01.02,C0040A'])
# for i in listt:
#     lenn = len(df_s08[df_s08['S.08.01.01.02,C0040A']==i])
#     if lenn > 1:
#         list_ind = list(df_s08.loc[df_s08['S.08.01.01.02,C0040A']==i].index)
#         temp = 0
#         for j in list_ind[1:]:
#             temp=temp+1
#             df_s08['S.08.01.01.02,C0040A'].iloc[j] = df_s08['S.08.01.01.02,C0040A'].iloc[j] + '_' + str(temp)
# df_s08 = df_s08.set_index(['entity', 'period', 'S.08.01.01.02,C0040A'])

In this tutorial we work with dummy data in order to show results

In [ ]:
df_s08_1 = pd.read_pickle(join(TEST_DATA_PATH,'S.08.01.01.01.pickle')).reset_index() #Import demo pickles
df_s08_2 = pd.read_pickle(join(TEST_DATA_PATH, 'S.08.01.01.02.pickle')).reset_index() #Import demo pickles
df_s08 = pd.merge(df_s08_1,df_s08_2,how='inner', left_on=['entity','period','S.08.01.01.01,C0040'], right_on=['entity','period','S.08.01.01.02,C0040']).set_index(['entity', 'period', 'S.08.01.01.01,C0040'])
df_s08_2 = df_s08_2.set_index(['entity', 'period', 'S.08.01.01.02,C0040'])

### Evaluate rules

Now we are ready to evaluate the different sets of rules. First, we construct a PatternMiner-object with the data-patterns package using the rules dataframe. Second, we use the analyze-function to get the results of the rules. We do this for each set of rules separately.

#### S06


In [ ]:
miner = data_patterns.PatternMiner(df_patterns=dfr_s06)
results_06 = miner.analyze(df_s06)
results_06

In [ ]:
miner = data_patterns.PatternMiner(df_patterns=dfr_s06_2)
results_06_2 = miner.analyze(df_s06_2)
results_06_2

In [ ]:
miner = data_patterns.PatternMiner(df_patterns=dfr_s06_1)
results_06_1 = miner.analyze(df_s06_1)
results_06_1

#### S08

In [ ]:
miner = data_patterns.PatternMiner(df_patterns=dfr_s08)
results_08 = miner.analyze(df_s08)
results_08

In [ ]:
miner2 = data_patterns.PatternMiner(df_patterns=dfr_s08_2)
results_08_2 = miner2.analyze(df_s08_2)
results_08_2